## Sambhav Agrawal 19264 Data Science and Engineering

## Question 6

## HOG features with SVM classification

In [7]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
# from keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import cv2

from skimage.feature import hog


from tqdm.notebook import tqdm_notebook as tqdm

import os

In [8]:
real = "../input/real-and-fake-face-detection/real_and_fake_face/training_real/"
fake = "../input/real-and-fake-face-detection/real_and_fake_face/training_fake/"

real_path = os.listdir(real)
fake_path = os.listdir(fake)

In [9]:
def load_img(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(64, 128))
    return image[...,::-1]

In [10]:
fig = plt.figure(figsize=(10, 10))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(load_img(real + real_path[i]), cmap='gray')
    plt.suptitle("Real faces",fontsize=20)
    plt.axis('off')

plt.show()


In [11]:
fig = plt.figure(figsize=(10,10))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(load_img(fake + fake_path[i]), cmap='gray')
    plt.suptitle("Fakes faces",fontsize=20)
    plt.title(fake_path[i][:4])
    plt.axis('off')

plt.show()

In [12]:
# dataset_path = "/kaggle/input/real-and-fake-face-detection/real_and_fake_face"


# pos = "../input/real-and-fake-face-detection/real_and_fake_face/training_real/"
# neg = "../input/real-and-fake-face-detection/real_and_fake_face/training_fake/"

# pos_path = os.listdir(pos)
# neg_path = os.listdir(neg)

In [13]:
def load_img(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(64, 128))
    fd, hog_image_fake = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize = True)
    hog_images_fake.append(hog_image_fake)
    hog_features_fake.append(fd)
    return image[...,::-1]

In [14]:
fig = plt.figure(figsize=(10,10))

ppc = 16
hog_images_fake = []
hog_features_fake = []

for i in range(960):
    plt.subplot(1000, 1000, i+1)
    plt.imshow(load_img(fake + fake_path[i]), cmap='gray')
    plt.suptitle("Fakes faces",fontsize=20)
    plt.title(fake_path[i][:1000])
    plt.axis('off')

plt.show()

In [15]:
plt.imshow(hog_images_fake[900])

In [16]:
def load_img(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image,(64, 128))
    fd, hog_image_real = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize = True)
    hog_images_real.append(hog_image_real)
    hog_features_real.append(fd)
    return image[...,::-1]

In [17]:
fig = plt.figure(figsize=(10,10))

ppc = 16
hog_images_real = []
hog_features_real = []

for i in range(1081):
    plt.subplot(1000, 1000, i+1)
    plt.imshow(load_img(real + real_path[i]), cmap='gray')
    plt.suptitle("Real faces",fontsize=20)
    plt.title(real_path[i][:1090])
    plt.axis('off')

plt.show()


In [18]:

plt.imshow(hog_images_real[1080])

In [19]:
from sklearn import svm
clf = svm.SVC()

hog_features_fake = np.array(hog_features_fake)
hog_features_real = np.array(hog_features_real)
data_frame_fake = pd.DataFrame(hog_features_fake)
data_frame_real = pd.DataFrame(hog_features_real)



In [20]:
# percentage = 80
# partition = int(len(hog_features_fake)*percentage/100)

# percentage = 80
# partition = int(len(hog_features_real)*percentage/100)

In [21]:
data_frame_fake.to_csv('X_fake.csv', index = False)

data_frame_real.to_csv('X_real.csv', index=False)


data_frame_real.shape

In [22]:
data_frame_fake.shape

In [23]:
X_fake = pd.read_csv('./X_fake.csv')
X_real = pd.read_csv('./X_real.csv')

X_fake['labels'] = 0

X_real['labels'] = 1

In [24]:
X_fake.shape

In [25]:
X_real.shape

In [26]:
frames = [X_fake, X_real]

X = pd.concat(frames)

In [27]:
X.shape

In [28]:
y = X['labels']


X.drop('labels', axis=1, inplace=True)


In [29]:
X.shape

In [30]:
y.shape

In [43]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [44]:
clf.fit(x_train,y_train)

In [45]:
y_pred = clf.predict(x_test)

In [46]:

from sklearn.metrics import classification_report,accuracy_score


print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))